In [2]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Master - Environment Setup

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/fraudfinder/raw/main/00_environment_setup.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Google Cloud Notebooks">Open in Cloud Notebook
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/fraudfinder/blob/main/00_environment_setup.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/fraudfinder/blob/main/00_environment_setup.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

[FraudFinder](https://github.com/googlecloudplatform/fraudfinder) is a series of labs on how to build a real-time fraud detection system on Google Cloud. Throughout the FraudFinder labs, you will learn how to read historical bank transaction data stored in data warehouse, read from a live stream of new transactions, perform exploratory data analysis (EDA), do feature engineering, ingest features into a feature store, train a model using feature store, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do real-time inference on your model with feature store, and monitor your model.

### Objective

Before you run this notebook, make sure that you have completed the steps in [README](README.md).

In this notebook, you will setup your environment for Fraudfinder to be used in subsequent labs.

This lab uses the following Google Cloud services and resources:

- [Vertex AI](https://cloud.google.com/vertex-ai/)
- [BigQuery](https://cloud.google.com/bigquery/)
- [Google Cloud Storage](https://cloud.google.com/storage)
- [Pub/Sub](https://cloud.google.com/pubsub/)

Steps performed in this notebook:

- Setup your environment.
- Load historical bank transactions into BigQuery.
- Read data from BigQuery tables.
- Read data from Pub/Sub topics, which contain a live stream of new transactions.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* Pub/Sub
* BigQuery

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), [Pub/Sub pricing](https://cloud.google.com/pubsub/pricing), [BigQuery pricing](https://cloud.google.com/bigquery/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Install additional packages

Install the following packages required to execute this notebook.

In [ ]:
!~/.venv/bin/pip install -U -r 'requirements.txt'
!~/.venv/bin/pip install -U google-cloud-aiplatform

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os


if not os.getenv("IS_TESTING"):
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Setup your environment

Run the next cells to import libraries used in this notebook and configure some options.

Run the next cell to set your project ID and some of the other constants used in the lab.  

### Replace REGION Below ***

<div class="alert alert-block alert-info">
<b>NOTE: REGION will be detected based on the Persistence Resrouces<br>
    You will need to run from this cell and below after Kernel restart
   </b>
</div>


In [ ]:
import json
import random
import string
from typing import Union

import pandas as pd
from google.cloud import bigquery

# Generate unique ID to help w/ unique naming of certain pieces
ID = "".join(random.choices(string.ascii_lowercase + string.digits, k=5))

# Project info and persistent Resource ID
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
PROJECT_NUM = !gcloud projects list --filter="$PROJECT_ID" --format="value(PROJECT_NUMBER)"
PROJECT_NUM = PROJECT_NUM[0]

# service account
SERVICE_ACCOUNT = f"{PROJECT_NUM}-compute@developer.gserviceaccount.com"

# Dynamically retrieve Persistent Resource location
PERSISTENT_RESOURCE_REGION = ""
check_regions = ["us-central1", "asia-southeast1", "europe-west4"]

for region in check_regions:
    shell_output = !gcloud ai persistent-resources list --project=$PROJECT_ID --region=$region
    if "Listed 0 items." not in shell_output:
        print(f"Persistent Resource found in {region}")
        PERSISTENT_RESOURCE_REGION = region

# Replace Region here
if PERSISTENT_RESOURCE_REGION:
    REGION = PERSISTENT_RESOURCE_REGION
else:
    REGION = "us-central1"

# GCS resource
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder-{ID}"
STAGING_BUCKET = f"{PROJECT_ID}-model-upload-{ID}"
AGENT_BUCKET = f"{PROJECT_ID}-ai-workshops"

# network
NETWORK = "fraud-finder-network"
SUBNET = "https://www.googleapis.com/compute/v1/projects/fraud-finder-lab/regions/us-central1/subnetworks/us-central1"

# pub/sub
SUBSCRIPTION_NAME = "ff-tx-for-feat-eng-sub"
SUBSCRIPTION_PATH = f"projects/{PROJECT_ID}/subscriptions/ff-tx-for-feat-eng-sub"

# Model training data info
RAW_BQ_TRANSACTION_TABLE_URI = f"{PROJECT_ID}.tx.tx"
RAW_BQ_LABELS_TABLE_URI = f"{PROJECT_ID}.tx.txlabels"
DATA_URI = "data"
TRAIN_DATA_URI = f"{DATA_URI}/train.csv"
READ_INSTANCES_TABLE = f"ground_truth_{ID}"
READ_INSTANCES_URI = f"bq://{PROJECT_ID}.tx.ground_truth_{ID}"
FEATURESTORE_ID = f"fraudfinder_{ID}"
FEATUREVIEW_ID = f"fraudfinder_view_{ID}"
DATASET_NAME = f"fraud_finder_dataset_{ID}"
BQ_DATASET = "tx"

# Model training info
ONLINE_STORAGE_NODES = 1
MODEL_REGISTRY = "ff_model"
MODEL_NAME = "ff_model"
EXPERIMENT_NAME = f"ff-experiment-{ID}"
JOB_NAME = f"fraudfinder-train-torch-{ID}"
TRAIN_COMPUTE = "e2-standard-4"
METRICS_URI = f"gs://{BUCKET_NAME}/deliverables/metrics.json"
AVG_PR_THRESHOLD = 0.2
MODEL_THRESHOLD = 0.5

# Model training data metadata
CUSTOMER_ENTITY_ID = "customer"
CUSTOMER_ENTITY_ID_FIELD = "customer_id"
TERMINAL_ENTITY_ID = "terminal"
TERMINALS_ENTITY_ID_FIELD = "terminal_id"
FEATURES_BQ_TABLE_URI = f"{PROJECT_ID}.tx.wide_features_table"
FEATURE_TIME = "feature_timestamp"
DROP_COLUMNS = [
    "timestamp",
]
FEAT_COLUMNS = [
    "customer_id_avg_amount_14day_window",
    "customer_id_avg_amount_15min_window",
    "customer_id_avg_amount_1day_window",
    "customer_id_avg_amount_30min_window",
    "customer_id_avg_amount_60min_window",
    "customer_id_avg_amount_7day_window",
    "customer_id_nb_tx_14day_window",
    "customer_id_nb_tx_15min_window",
    "customer_id_nb_tx_1day_window",
    "customer_id_nb_tx_30min_window",
    "customer_id_nb_tx_60min_window",
    "customer_id_nb_tx_7day_window",
    "terminal_id_avg_amount_15min_window",
    "terminal_id_avg_amount_30min_window",
    "terminal_id_avg_amount_60min_window",
    "terminal_id_nb_tx_14day_window",
    "terminal_id_nb_tx_15min_window",
    "terminal_id_nb_tx_1day_window",
    "terminal_id_nb_tx_30min_window",
    "terminal_id_nb_tx_60min_window",
    "terminal_id_nb_tx_7day_window",
    "terminal_id_risk_14day_window",
    "terminal_id_risk_1day_window",
    "terminal_id_risk_7day_window",
    "tx_amount",
]
TARGET_COLUMN = "tx_fraud"
DATA_SCHEMA = {
    "timestamp": "object",
    "tx_amount": "float64",
    "tx_fraud": "Int64",
    "entity_type_customer": "Int64",
    "customer_id_nb_tx_1day_window": "Int64",
    "customer_id_nb_tx_7day_window": "Int64",
    "customer_id_nb_tx_14day_window": "Int64",
    "customer_id_avg_amount_1day_window": "float64",
    "customer_id_avg_amount_7day_window": "float64",
    "customer_id_avg_amount_14day_window": "float64",
    "customer_id_nb_tx_15min_window": "Int64",
    "customer_id_avg_amount_15min_window": "float64",
    "customer_id_nb_tx_30min_window": "Int64",
    "customer_id_avg_amount_30min_window": "float64",
    "customer_id_nb_tx_60min_window": "Int64",
    "customer_id_avg_amount_60min_window": "float64",
    "entity_type_terminal": "Int64",
    "terminal_id_nb_tx_1day_window": "Int64",
    "terminal_id_nb_tx_7day_window": "Int64",
    "terminal_id_nb_tx_14day_window": "Int64",
    "terminal_id_risk_1day_window": "float64",
    "terminal_id_risk_7day_window": "float64",
    "terminal_id_risk_14day_window": "float64",
    "terminal_id_nb_tx_15min_window": "Int64",
    "terminal_id_avg_amount_15min_window": "float64",
    "terminal_id_nb_tx_30min_window": "Int64",
    "terminal_id_avg_amount_30min_window": "float64",
    "terminal_id_nb_tx_60min_window": "Int64",
    "terminal_id_avg_amount_60min_window": "float64",
}

# Model deployment info
ENDPOINT_NAME = "ff_model_endpoint"
IMAGE_REPOSITORY = f"fraudfinder-{ID}"
IMAGE_NAME = "torch-classifier"
IMAGE_TAG = "latest"
IMAGE_URI = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{IMAGE_REPOSITORY}/{IMAGE_NAME}:{IMAGE_TAG}"
SERVING_IMAGE_NAME = "torch-fastapi-server"
MODEL_SERVING_IMAGE_URI = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{IMAGE_REPOSITORY}/{SERVING_IMAGE_NAME}:{IMAGE_TAG}"
DEPLOY_COMPUTE = "n1-standard-4"
BASE_IMAGE = "python:3.11"
PIPELINE_NAME = f"fraud-finder-torch-pipeline-{ID}"
PIPELINE_ROOT = f"gs://{BUCKET_NAME}/pipelines"
AVG_PR_CONDITION = "avg_pr_condition"
PERSISTENT_RESOURCE_ID = "ai-takeoff" if PERSISTENT_RESOURCE_REGION else None
REPLICA_COUNT = 1

In [2]:

config = {
    "PROJECT_ID": PROJECT_ID,
    "BUCKET_NAME": BUCKET_NAME,
    "STAGING_BUCKET": STAGING_BUCKET,
    "REGION": REGION,
    "ID": ID,
    "CUSTOMER_ENTITY_ID": CUSTOMER_ENTITY_ID,
    "CUSTOMER_ENTITY_ID_FIELD": CUSTOMER_ENTITY_ID_FIELD,
    "TERMINAL_ENTITY_ID": TERMINAL_ENTITY_ID,
    "TERMINALS_ENTITY_ID_FIELD": TERMINALS_ENTITY_ID_FIELD,
    "FEATURESTORE_ID": FEATURESTORE_ID,
    "FEATUREVIEW_ID": FEATUREVIEW_ID,
    "NETWORK": NETWORK,
    "SUBNET": SUBNET,
    "MODEL_REGISTRY": MODEL_REGISTRY,
    "RAW_BQ_TRANSACTION_TABLE_URI": RAW_BQ_TRANSACTION_TABLE_URI,
    "RAW_BQ_LABELS_TABLE_URI": RAW_BQ_LABELS_TABLE_URI,
    "FEATURES_BQ_TABLE_URI": FEATURES_BQ_TABLE_URI,
    "FEATURE_TIME": FEATURE_TIME,
    "ONLINE_STORAGE_NODES": ONLINE_STORAGE_NODES,
    "SUBSCRIPTION_NAME": SUBSCRIPTION_NAME,
    "SUBSCRIPTION_PATH": SUBSCRIPTION_PATH,
    "DROP_COLUMNS": DROP_COLUMNS,
    "FEAT_COLUMNS": FEAT_COLUMNS,
    "TARGET_COLUMN": TARGET_COLUMN,
    "DATA_SCHEMA": DATA_SCHEMA,
    "MODEL_NAME": MODEL_NAME,
    "EXPERIMENT_NAME": EXPERIMENT_NAME,
    "DATA_URI": DATA_URI,
    "TRAIN_DATA_URI": TRAIN_DATA_URI,
    "READ_INSTANCES_TABLE": READ_INSTANCES_TABLE,
    "READ_INSTANCES_URI": READ_INSTANCES_URI,
    "DATASET_NAME": DATASET_NAME,
    "JOB_NAME": JOB_NAME,
    "ENDPOINT_NAME": ENDPOINT_NAME,
    "MODEL_SERVING_IMAGE_URI": MODEL_SERVING_IMAGE_URI,
    "IMAGE_REPOSITORY": IMAGE_REPOSITORY,
    "IMAGE_NAME": IMAGE_NAME,
    "IMAGE_TAG": IMAGE_TAG,
    "IMAGE_URI": IMAGE_URI,
    "TRAIN_COMPUTE": TRAIN_COMPUTE,
    "DEPLOY_COMPUTE": DEPLOY_COMPUTE,
    "BASE_IMAGE": BASE_IMAGE,
    "PIPELINE_NAME": PIPELINE_NAME,
    "PIPELINE_ROOT": PIPELINE_ROOT,
    "BQ_DATASET": BQ_DATASET,
    "METRICS_URI": METRICS_URI,
    "AVG_PR_THRESHOLD": AVG_PR_THRESHOLD,
    "MODEL_THRESHOLD": MODEL_THRESHOLD,
    "AVG_PR_CONDITION": AVG_PR_CONDITION,
    "PERSISTENT_RESOURCE_ID": PERSISTENT_RESOURCE_ID,
    "REPLICA_COUNT": REPLICA_COUNT,
    "SERVICE_ACCOUNT": SERVICE_ACCOUNT,
}

print(json.dumps(config, indent=2))

{
  "PROJECT_ID": "ai-takeoff-2025",
  "BUCKET_NAME": "ai-takeoff-2025-fraudfinder-lh9ca",
  "STAGING_BUCKET": "ai-takeoff-2025-model-upload-lh9ca",
  "REGION": "us-central1",
  "ID": "lh9ca",
  "CUSTOMER_ENTITY_ID": "customer",
  "CUSTOMER_ENTITY_ID_FIELD": "customer_id",
  "TERMINAL_ENTITY_ID": "terminal",
  "TERMINALS_ENTITY_ID_FIELD": "terminal_id",
  "FEATURESTORE_ID": "fraudfinder_lh9ca",
  "FEATUREVIEW_ID": "fraudfinder_view_lh9ca",
  "NETWORK": "fraud-finder-network",
  "SUBNET": "https://www.googleapis.com/compute/v1/projects/fraud-finder-lab/regions/us-central1/subnetworks/us-central1",
  "MODEL_REGISTRY": "ff_model",
  "RAW_BQ_TRANSACTION_TABLE_URI": "ai-takeoff-2025.tx.tx",
  "RAW_BQ_LABELS_TABLE_URI": "ai-takeoff-2025.tx.txlabels",
  "FEATURES_BQ_TABLE_URI": "ai-takeoff-2025.tx.wide_features_table",
  "FEATURE_TIME": "feature_timestamp",
  "ONLINE_STORAGE_NODES": 1,
  "SUBSCRIPTION_NAME": "ff-tx-for-feat-eng-sub",
  "SUBSCRIPTION_PATH": "projects/ai-takeoff-2025/subscripti

In [11]:
from google.cloud import storage


def create_bucket(bucket_name: str, location: str) -> None:
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    _ = client.create_bucket(bucket, location=location)


create_bucket(config["BUCKET_NAME"], config["REGION"])
create_bucket(config["STAGING_BUCKET"], config["REGION"])

In [12]:
!gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin gs://{BUCKET_NAME}
!gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin gs://{STAGING_BUCKET}

In [5]:
import yaml


def write_to_bucket(bucket_name: str, uri: str, content: str | dict | list) -> None:
    client = storage.Client()
    bucket = client.bucket(bucket_name=bucket_name)
    blob = bucket.blob(uri)
    blob.upload_from_string(yaml.safe_dump(content))


write_to_bucket(config["BUCKET_NAME"], f"config/vertex_conf_{config['ID']}.yaml", config)

with open("config_path.json", "w") as f:
    json.dump({"bucket": config["BUCKET_NAME"], "conf_uri": f"config/vertex_conf_{config['ID']}.yaml"}, f, indent=2)

### Copy the historical transaction data into BigQuery tables

Now we will copy the historical transaction data and ingest it into BigQuery tables. For this, we will need to run `copy_bigquery_data.py`.

In [18]:
!python3 scripts/copy_bigquery_data.py $BUCKET_NAME $REGION $PROJECT_ID

ai-takeoff-2025-fraudfinder-lh9ca us-central1 ai-takeoff-2025
File copied from gs://cymbal-fraudfinder/datagen/hacked_customers_history.txt 
		 to gs://ai-takeoff-2025-fraudfinder-lh9ca/datagen/hacked_customers_history.txt
File copied from gs://cymbal-fraudfinder/datagen/hacked_terminals_history.txt 
		 to gs://ai-takeoff-2025-fraudfinder-lh9ca/datagen/hacked_terminals_history.txt
File copied from gs://cymbal-fraudfinder/datagen/demographics/customer_profiles.csv 
		 to gs://ai-takeoff-2025-fraudfinder-lh9ca/datagen/demographics/customer_profiles.csv
File copied from gs://cymbal-fraudfinder/datagen/demographics/terminal_profiles.csv 
		 to gs://ai-takeoff-2025-fraudfinder-lh9ca/datagen/demographics/terminal_profiles.csv
File copied from gs://cymbal-fraudfinder/datagen/demographics/customer_with_terminal_profiles.csv 
		 to gs://ai-takeoff-2025-fraudfinder-lh9ca/datagen/demographics/customer_with_terminal_profiles.csv
BigQuery table created: `ai-takeoff-2025`.tx.tx
BigQuery table create

### Check data in BigQuery

After ingesting our data into BigQuery, it's time to run some queries against the tables to inspect the data. You can also go to the [BigQuery console](https://console.cloud.google.com/bigquery) to see the data.

#### Initialize BigQuery SDK for Python 

Use a helper function for sending queries to BigQuery.

In [36]:
# Wrapper to use BigQuery client to run query/job, return job ID or result as DF
def run_bq_query(sql: str) -> Union[str, pd.DataFrame]:
    """
    Run a BigQuery query and return the job ID or result as a DataFrame
    Args:
        sql: SQL query, as a string, to execute in BigQuery
    Returns:
        df: DataFrame of results from query,  or error, if any
    """

    bq_client = bigquery.Client()
    

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")
    return df

#### tx.tx
The `tx.tx` table contains the basic information about each transaction:
- `TX_ID` is a unique ID per transaction
- `TX_TS` is the timestamp of the transaction, in UTC
- `CUSTOMER_ID` is a unique 16-digit string ID per customer
- `TERMINAL_ID` is a unique 16-digit string ID per point-of-sale terminal
- `TX_AMOUNT` is the amount of money spent by the customer at a terminal, in dollars

In [49]:
run_bq_query(
    """
SELECT
  *
FROM
  tx.tx
LIMIT 5
"""
)

Finished job_id: a00a2604-f875-4bd0-8963-f196c5116bd3


,TX_ID,TX_TS,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,b837607fc4681fde61e8191ff70b45aa9cf1b6f1,2025-03-03 12:22:26+00:00,4428639496183819,00064542,76.850000000
1,080a509dc486d75323e6339a4b6eb5699d7a58e7,2025-03-03 19:53:54+00:00,9034584161813968,00064542,25.480000000
2,fd01bb15e8c6bf6788ef59374bbb0bbd8db07d41,2025-03-03 17:04:27+00:00,3503905935018327,00064542,94.420000000
3,abcbf07b40a4b6c4562b135642c5df806bff0e65,2025-03-03 04:44:46+00:00,0717942689361348,00064542,76.960000000
4,014c046c8fcb069fd4ec6cc464c3c90e7c8cff41,2025-03-03 06:32:12+00:00,6580876822931629,00064542,59.090000000


#### tx.txlabels
The `tx.txlabels` table contains information on whether each transation was fraud or not:
- `TX_ID` is a unique ID per transaction
- `TX_FRAUD` is 1 if the transaction was fraud, and 0 if the transaction was not fraudulent

In [50]:
run_bq_query(
    """
SELECT
  *
FROM
  tx.txlabels
LIMIT 5
"""
)

Finished job_id: f65fa0a6-e66e-4371-af33-808f20285e84


,TX_ID,TX_FRAUD
0,10acd92a057851af33982987c368506b231dd006,0
1,9d3329d73b49c62b99e7922170692845916f9dd6,0
2,f918c8e12b7b5e58e4a940d3bea7b955484f62b6,0
3,2a89a48489e7b7680a4f93f9ecdbf065469f399a,0
4,5ae74feb01aa582c3ccfca6acc50635c19fdefe7,0


### Check live streaming transactions via public Pub/Sub topics

As part of the [README](README.md), you've created [subscriptions](https://console.cloud.google.com/cloudpubsub/subscription/) to public Pub/Sub topics, where there is a constant flow of new transactions. This means you have, in your own Google Cloud project, subscriptions to the public Pub/Sub topics. You will receive a Pub/Sub message in your subscription every time a new transaction is streamed into the Pub/Sub topic.

There are two public Pub/Sub topics where there is a constant stream of live transactions occurring.

The following Pub/Sub topics are used for transactions:
```
projects/cymbal-fraudfinder/topics/ff-tx
projects/cymbal-fraudfinder/topics/ff-txlabels
```

Note: If you haven't completed the steps in the README, please make sure that you complete them first before continuing this notebook, otherwise you may not have Pub/Sub subscriptions.